### RUN ONLY ONCE
from ctgan.utils import save_real_df, save_fake_df, ohe_cat
for dset in ['loan']:
    save_real_df(dset)
    save_fake_df(dset)
    ohe_cat(dset)

In [ ]:
import json

from evaluation import get_utility_metrics, stat_sim
from ctgan.utils import BASE_DIR, get_cols

AE_TYPES = ['vanilla', 'denoising', 'vae', 'ee']
PATHS = {
    'adult': {
        'real' : {
            'train': f"{BASE_DIR}/dataset/adult/real_data.csv",
            'test': f"{BASE_DIR}/dataset/adult/real_test.csv"
        },
        'fake' : {
            'train': {
                'vanilla' : f"{BASE_DIR}/dataset/adult/fake_data_vanilla.csv",
                'denoising' : f"{BASE_DIR}/dataset/adult/fake_data_denoising.csv",
                'vae' : f"{BASE_DIR}/dataset/adult/fake_data_vae.csv",
                'ee' : f"{BASE_DIR}/dataset/adult/fake_data_ee.csv"
            },
            'test': {
                'vanilla' : f"{BASE_DIR}/dataset/adult/fake_test_vanilla.csv",
                'denoising' : f"{BASE_DIR}/dataset/adult/fake_test_denoising.csv",
                'vae' : f"{BASE_DIR}/dataset/adult/fake_test_vae.csv",
                'ee' : f"{BASE_DIR}/dataset/adult/fake_test_ee.csv"
            }
        }
    }
}

with open(f"{BASE_DIR}/results/stat_sim.json", 'r') as f:
    STAT_SIM = json.load(f)

with open(f"{BASE_DIR}/results/ml_efficacy.json", 'r') as f:
    ML_EFFICACY = json.load(f)

## Adult

#### ML Efficacy

for ae_type in AE_TYPES:
    try:
        ML_EFFICACY['adult'][ae_type] = get_utility_metrics(real_path=PATHS['adult']['real']['test'],
                                                        fake_paths=[PATHS['adult']['fake']['test'][ae_type]]).tolist()
    except Exception as e:
        print(f"Mode Collapse Detected for AE_TYPE: {ae_type}")
        print(e)

with open(f"{BASE_DIR}/results/ml_efficacy.json", 'w') as f:
    f.write(json.dumps(ML_EFFICACY))

#### Statistical Similarity

In [ ]:
out = get_cols("adult")

for ae_type in AE_TYPES:
    STAT_SIM['adult'][ae_type] = stat_sim(real_path=PATHS['adult']['real']['train'],
                                          fake_path=PATHS['adult']['fake']['train'][ae_type],
                                          cat_cols=out['discrete_cols'][1])

with open(f"{BASE_DIR}/results/stat_sim.json", 'w') as f:
    f.write(json.dumps(STAT_SIM))